In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10201919966866310855
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14328594432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10095377893787187263
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
import os
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import ShuffleSplit

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)

set_all_seeds(420)

In [ ]:
model_mn2 = MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)

In [ ]:
model_mn2.summary()

In [ ]:
layer_name = 'block_15_add'
feature_extractor = Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

## Training

In [ ]:
plants_names = []

for plant in os.listdir('gdrive/MyDrive/flower_data/encoded/train'):
    plants_names += [plant.split('.npy')[0]]

plants_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(plants_names)}

In [ ]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f"gdrive/MyDrive/flower_data/encoded/{set_kind}")):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f"gdrive/MyDrive/flower_data/encoded/{set_kind}/{file}")
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[101] val/16.npy

In [ ]:
n_classes = len(plants_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]


X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [ ]:
input_layer = tf.keras.layers.Input(shape=[7, 7, 160], name="submodel_inputs")
intermediate = tf.keras.layers.Conv2D(filters=960, kernel_size=1, use_bias=False, name="block_16_expand")(input_layer)
intermediate = tf.keras.layers.BatchNormalization(name="block_16_expand_BN")(intermediate)
intermediate = tf.keras.layers.ReLU(name="block_16_expand_relu")(intermediate)
intermediate = tf.keras.layers.DepthwiseConv2D(
    kernel_size=3, padding='same', use_bias=False, name="block_16_depthwise")(intermediate)
intermediate = tf.keras.layers.BatchNormalization(name="block_16_depthwise_BN")(intermediate)
intermediate = tf.keras.layers.ReLU(name="block_16_depthwise_relu")(intermediate)
intermediate = tf.keras.layers.Conv2D(filters=320, kernel_size=1, use_bias=False, name="block_16_project")(intermediate)
intermediate = tf.keras.layers.BatchNormalization(name="block_16_project_BN")(intermediate)
intermediate = tf.keras.layers.Conv2D(filters=1280, kernel_size=1, use_bias=False, name="Conv_1")(intermediate)
intermediate = tf.keras.layers.BatchNormalization(name="Conv_1_bn")(intermediate)
intermediate = tf.keras.layers.ReLU(name="out_relu")(intermediate)
intermediate = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling2d")(intermediate)
intermediate = tf.keras.layers.Dropout(rate=0.2, name="dropout")(intermediate)
output_layer = tf.keras.layers.Dense(
    units=102, activation="softmax", kernel_regularizer=keras.regularizers.l1(0.01), name="predictions")(intermediate)

model = Model(input_layer, output_layer)
model.summary()

for layer in model.layers:
    if layer.name not in ["submodel_inputs", "dropout", "predictions"]:
        layer.set_weights(model_mn2.get_layer(layer.name).get_weights())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 submodel_inputs (InputLayer  [(None, 7, 7, 160)]      0         
 )                                                               
                                                                 
 block_16_expand (Conv2D)    (None, 7, 7, 960)         153600    
                                                                 
 block_16_expand_BN (BatchNo  (None, 7, 7, 960)        3840      
 rmalization)                                                    
                                                                 
 block_16_expand_relu (ReLU)  (None, 7, 7, 960)        0         
                                                                 
 block_16_depthwise (Depthwi  (None, 7, 7, 960)        8640      
 seConv2D)                                                       
                                                           

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)

In [ ]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=1)

In [ ]:
idx = np.arange(X_train.shape[0])
np.random.shuffle(idx)

X_train_shuffled = X_train[idx]
y_train_shuffled = y_train[idx]

# Free some precious RAM
del X_train
del instances['train']

In [ ]:
checkpoint = ModelCheckpoint(filepath='model_mobilenetv2_regularized_oxford102',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

history = model.fit(
    X_train_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)

Epoch 1/10
332/333 [============================>.] - ETA: 0s - loss: 7.1402 - accuracy: 0.7476
Epoch 1: val_loss improved from inf to 1.75924, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 17s 23ms/step - loss: 7.1358 - accuracy: 0.7474 - val_loss: 1.7592 - val_accuracy: 0.8908 - lr: 0.0010
Epoch 2/10
329/333 [============================>.] - ETA: 0s - loss: 1.6692 - accuracy: 0.9247
Epoch 2: val_loss improved from 1.75924 to 1.39664, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 6s 18ms/step - loss: 1.6682 - accuracy: 0.9245 - val_loss: 1.3966 - val_accuracy: 0.9300 - lr: 0.0010
Epoch 3/10
333/333 [==============================] - ETA: 0s - loss: 1.3842 - accuracy: 0.9516
Epoch 3: val_loss improved from 1.39664 to 1.29316, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 6s 17ms/step - loss: 1.3842 - accuracy: 0.9516 - val_loss: 1.2932 - val_accuracy: 0.9566 - lr: 0.0010
Epoch 4/10
331/333 [============================>.] - ETA: 0s - loss: 1.2497 - accuracy: 0.9680
Epoch 4: val_loss improved from 1.29316 to 1.21352, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 6s 19ms/step - loss: 1.2504 - accuracy: 0.9679 - val_loss: 1.2135 - val_accuracy: 0.9426 - lr: 0.0010
Epoch 5/10
329/333 [============================>.] - ETA: 0s - loss: 1.1750 - accuracy: 0.9726
Epoch 5: val_loss improved from 1.21352 to 1.13055, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 5s 16ms/step - loss: 1.1749 - accuracy: 0.9727 - val_loss: 1.1306 - val_accuracy: 0.9580 - lr: 0.0010
Epoch 6/10
333/333 [==============================] - ETA: 0s - loss: 1.1340 - accuracy: 0.9736
Epoch 6: val_loss did not improve from 1.13055
333/333 [==============================] - 4s 11ms/step - loss: 1.1340 - accuracy: 0.9736 - val_loss: 1.1717 - val_accuracy: 0.9482 - lr: 0.0010
Epoch 7/10
329/333 [============================>.] - ETA: 0s - loss: 0.8798 - accuracy: 0.9861
Epoch 7: val_loss improved from 1.13055 to 0.84200, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 6s 19ms/step - loss: 0.8813 - accuracy: 0.9860 - val_loss: 0.8420 - val_accuracy: 0.9692 - lr: 1.0000e-04
Epoch 8/10
330/333 [============================>.] - ETA: 0s - loss: 0.8315 - accuracy: 0.9905
Epoch 8: val_loss improved from 0.84200 to 0.82442, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 5s 16ms/step - loss: 0.8323 - accuracy: 0.9905 - val_loss: 0.8244 - val_accuracy: 0.9678 - lr: 1.0000e-04
Epoch 9/10
332/333 [============================>.] - ETA: 0s - loss: 0.8150 - accuracy: 0.9922
Epoch 9: val_loss improved from 0.82442 to 0.81460, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 6s 19ms/step - loss: 0.8164 - accuracy: 0.9920 - val_loss: 0.8146 - val_accuracy: 0.9664 - lr: 1.0000e-04
Epoch 10/10
328/333 [============================>.] - ETA: 0s - loss: 0.8096 - accuracy: 0.9930
Epoch 10: val_loss improved from 0.81460 to 0.80766, saving model to model_mobilenetv2_regularized_oxford102


333/333 [==============================] - 6s 17ms/step - loss: 0.8112 - accuracy: 0.9929 - val_loss: 0.8077 - val_accuracy: 0.9748 - lr: 1.0000e-04


In [ ]:
model.save("model_mobilenetv2_regularized_oxford102")

In [ ]:
!zip -r model_mobilenetv2_regularized.zip model_mobilenetv2_regularized_oxford102/

  adding: model_mobilenetv2_regularized_oxford102/ (stored 0%)
  adding: model_mobilenetv2_regularized_oxford102/fingerprint.pb (stored 0%)
  adding: model_mobilenetv2_regularized_oxford102/variables/ (stored 0%)
  adding: model_mobilenetv2_regularized_oxford102/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: model_mobilenetv2_regularized_oxford102/variables/variables.index (deflated 67%)
  adding: model_mobilenetv2_regularized_oxford102/keras_metadata.pb (deflated 92%)
  adding: model_mobilenetv2_regularized_oxford102/assets/ (stored 0%)
  adding: model_mobilenetv2_regularized_oxford102/saved_model.pb (deflated 89%)


In [ ]:
def top_k_accuracy_score(y_true, y_pred, k=1):
    ranks = np.sum(y_pred >= y_pred[y_true == 1].reshape(-1, 1), axis=1)
    return np.sum(ranks <= k) / ranks.shape[0]

model = keras.models.load_model('model_mobilenetv2_regularized_oxford102')
print("Validation set accuracy")
preds_valid_1 = model(tf.reshape(X_valid, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_1, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_1, k=5))

print("Test set")
preds_test_aug_1 = model(tf.reshape(X_test, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_1, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_1, k=5))

Validation set accuracy
TOP 1 0.9747899159663865
TOP 5 0.9957983193277311
Test set
TOP 1 0.9572127139364304
TOP 5 0.9902200488997555
